In [1]:
import genjax
import jax
import jax.numpy as jnp
from genjax.core.interpreters import context
import dataclasses
from genjax.typing import Any
from genjax import param, trace, Normal

key = jax.random.PRNGKey(314159)
console = genjax.pretty()

In [2]:
@genjax.gen
def _submodel():
    y1 = genjax.trace("y1", genjax.Normal)(0.0, 1.0)
    y2 = genjax.trace("y2", genjax.Normal)(0.0, 1.0)
    return y1, y2


@genjax.gen
def hierarchical_simple_normal_multiple_returns():
    y1, y2 = genjax.trace("y", _submodel)()
    return y1, y2

In [3]:
fn = hierarchical_simple_normal_multiple_returns.simulate
key, tr = fn(key, ())
chm = tr.get_choices()
chm

TrieChoiceMap
└── trie
    └── Trie
        └── :y
            └── BuiltinTrace
                ├── gen_fn
                │   └── BuiltinGenerativeFunction
                │       └── source
                │           └── PytreeClosure
                │               ├── callable
                │               │   └── <function _submodel>
                │               └── environment
                │                   └── list
                ├── args
                │   └── tuple
                ├── retval
                │   └── tuple
                │       ├──  f32[]
                │       └──  f32[]
                ├── choices
                │   └── Trie
                │       ├── :y1
                │       │   └── DistributionTrace
                │       │       ├── gen_fn
                │       │       │   └── _Normal
                │       │       ├── args
                │       │       │   └── tuple
                │       │       │       ├── (const) 0.0
                │       │       │       └── (const) 1.0
                │       │       ├── value
                │       │       │   └──  f32[]
                │       │       └── score
                │       │           └──  f32[]
                │       └── :y2
                │           └── DistributionTrace
                │               ├── gen_fn
                │               │   └── _Normal
                │               ├── args
                │               │   └── tuple
                │               │       ├── (const) 0.0
                │               │       └── (const) 1.0
                │               ├── value
                │               │   └──  f32[]
                │               └── score
                │                   └──  f32[]
                ├── cache
                │   └── Trie
                └── score
                    └──  f32[]

In [4]:
new_chm = genjax.choice_map({("y", "y1"): 0.5})

In [5]:
key, (rd, w, tr, _) = tr.update(key, new_chm, ())
rd

(Diff(primal=0.5, tangent=), Diff(primal=Array(0.27442896, dtype=float32), tangent=))

In [6]:
rd[0]

Diff
├── primal
│   └── (const) 0.5
└── tangent
    └── _UnknownChange

In [7]:
w

Array(-0.11914301, dtype=float32)